<a href="https://colab.research.google.com/github/layolu/gpt-2/blob/fromscratch/jawiki_GPT_2_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-2 日本語版Wikipedia 学習済みモデル (117M) デモNotebook
作成者: layolu https://twitter.com/layolu


一時期「危険すぎる」とのことで公開が見送られていたなど巷で話題の OpenAI の文書生成モデル "GPT-2" 
([公式記事](https://openai.com/blog/better-language-models/), [参考記事1](https://japan.zdnet.com/article/35145106/), [参考記事2](https://gigazine.net/news/20191106-gpt-2-final-model-release/)
について、日本語での学習済みモデルが見当たらなかったため、2019/11の日本語版Wikipediaの全記事データで学習させたものを公開します。モデルサイズは117M (OpenAIが最初に公開したいわゆるsmallモデルと同様) です。

またこのColabノートブックでは、数回クリックすることでかんたんに、しかも無料でこのモデルを試すことができます。

## 学習済みモデルのダウンロード
[Google Drive](https://drive.google.com/file/d/1NpZcg_cVZ-WkjGOH14LFzg70196KRA6f/view?usp=sharing)で公開しています (1.7GB)。
このノートブックで試す限りはダウンロードは不要です。

## デモの遊び方
1. 左上の [PLAYGROUNDで開く] をクリックします。Googleへログインを求められた場合ログインします。
1. メニューの [ランタイム] → [すべてのセルを実行] をクリックすると、警告ダイアログが表示されるため [そのまま実行]をクリックします。自動でモデルのダウンロードはじめ環境のセットアップが行われます (数分かかります)。<br>
セットアップが完了すると、一番下の "入力フォーム" のセルにフォームが表示されます。
1. テキストエリアに文章を入力し "続きを生成" ボタンを押すと、GPT-2がその続きを自動生成し、結果が下に表示されます (CPU環境で50秒ほどかかります)。<br>

## TIPS
* 学習データ内で記事間の区切りに `<|endoftext|>` という文字列を使う仕様のため、入力として `<|endoftext|>` を与えるとランダムなWikipediaのフェイク記事が作成されます。
* また、`<|endoftext|>` の後に続けて文章を書くと、そこからWikipedia風の文章が作成されます (内容については全く保証できませんが）。<br>
おすすめとして、 `<|endoftext|>『` を与えると色々なジャンルの架空の作品やテレビ番組等の記事が頭から自動作成されます (内容については(略))。

## 利用したコード
* https://github.com/nshepperd/gpt-2/
OpenAIのオリジナルコードをもとに学習可能にしたフォーク
* https://github.com/rkfg/gpt-2/ 上記 nshepperd さんフォークから、さらに多国語対応のため[Sentencepiece](https://github.com/google/sentencepiece)に対応したフォーク

学習は rkfg さんのフォークで行いましたが、nshepperd さんフォークにはより自然な出力ができる top_p という実装が入っているため ([参考](https://www.gwern.net/GPT-2#training-gpt-2-poetry))、このデモではそのコードを rkfg さんフォークに組み込んだものを使っています ( https://github.com/layolu/gpt-2/ )。

Sentencepieceを利用しているため、このモデルは rkfg さんのフォーク（及び私のフォーク）でのみ動作します。


In [0]:
#@title モデルのダウンロードとソフトウェアのインストール
%%bash
[ ! -f ready ] && \
    curl gdrive.sh | bash -s "https://drive.google.com/file/d/1NpZcg_cVZ-WkjGOH14LFzg70196KRA6f/view?usp=sharing" && \
    unzip jawiki-117M-292612.zip && \
    mkdir models && \
    mv jawiki-117M-292612 models && \
    git clone https://github.com/layolu/gpt-2 && \
    pip install sentencepiece fire && \
    touch ready

In [0]:
#@title Pythonパッケージのインポート
%tensorflow_version 1.x 
import IPython
from google.colab import output as colab_output
import fire
import json
import os
import numpy as np
import tensorflow as tf
import sys
sys.path.append('gpt-2/src')
import model, sample, encoder_sp as encoder

In [0]:
#@title 学習済みモデルのローディング
#from https://github.com/nshepperd/gpt-2/blob/finetuning/src/interactive_conditional_samples.py
# and https://github.com/rkfg/gpt-2/blob/fromscratch/src/interactive_conditional_samples.py
model_name = 'jawiki-117M-292612'
seed = None
nsamples = 1
batch_size = 1
length = None
temperature =1
top_k = 0
top_p = 0.9

assert nsamples % batch_size == 0

enc = encoder.get_encoder(model_name)
hparams = model.default_hparams()
with open(os.path.join('models', model_name, 'hparams.json')) as f:
    hparams.override_from_dict(json.load(f))

if length is None:
    length = hparams.n_ctx // 2
elif length > hparams.n_ctx:
    raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

sess = tf.InteractiveSession()
context = tf.placeholder(tf.int32, [batch_size, None])
np.random.seed(seed)
tf.set_random_seed(seed)
output = sample.sample_sequence(
    hparams=hparams, length=length,
    context=context,
    batch_size=batch_size,
    temperature=temperature, top_k=top_k, top_p=top_p
)
saver = tf.train.Saver()
ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name, 'checkpoint/run1'))
saver.restore(sess, ckpt)


In [0]:
#@title テキスト生成用関数
def generate_text(input_text):
    context_tokens = enc.encode(input_text.replace('\n', '<|n|>'))
    generated = 0
    for _ in range(nsamples // batch_size):
        out = sess.run(output, feed_dict={
            context: [context_tokens for _ in range(batch_size)]
        })[:, len(context_tokens):]
        for i in range(batch_size):
            generated += 1
            gen_text = enc.decode(out[i])
        html = f'<strong><u>{input_text}</u></strong>{gen_text}'
        html = html.replace('\n', '<br>')
        html = html.replace('<|endoftext|>', '<hr>')
    return IPython.display.HTML(f'<p>{html}</p>')
colab_output.register_callback('notebook.generate', generate_text)

In [0]:
#@title 入力フォーム
%%html
<div name="gpt-2-interface">
    <textarea id="gpt-2-context" rows="6" cols="160">おいしいカレーの作り方は、</textarea>
    <br>
    <button id="gpt-2-generate" onclick="generate()">続きを生成</button>
    <div id="gpt-2-result"></div>
</div>
<script>
async function generate() {
    console.time('generate')
    document.querySelector("#gpt-2-generate").disabled = true;
    const context = document.querySelector("#gpt-2-context").value;
    console.log(context)
    const result = await google.colab.kernel.invokeFunction(
    'notebook.generate', // The callback name.
    [context], // The arguments.
    {}); // kwargs
    console.dir(result);
    const html = result.data['text/html'];
    document.querySelector("#gpt-2-result").innerHTML = html;
    document.querySelector("#gpt-2-generate").disabled = false;
    console.timeEnd('generate')
};
</script>